In [32]:
from pathlib import Path
import os

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE','')

if iskaggle:
    path = Path('../input/titanic')
    ! pip install -Uqq fastai

else:
    import zipfile, kaggle
    path = Path('titanic')
    if not path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

In [33]:
from fastai.tabular.all import *

pd.options.display.float_format = '{:.2f}'.format
set_seed(42)

In [34]:
df = pd.read_csv(path/'train.csv')

In [35]:
def add_feature(df):
    df['logFare'] = np.log1p(df['Fare'])
    df['Deck'] = df.Cabin.str[0].map(dict(A='ABC', B='ABC', C='ABC', D='DE', E='FG', G='FG'))
    df['Family'] = df.SibSp + df.Parch
    df['Alone'] = df.Family == 0
    df['TicketFreq'] = df.groupby('Ticket')['Ticket'].transform('count')
    df['Title'] = df.Name.str.split(', ', expand = True)[1].str.split('.', expand = True)[0]
    df['Title'] = df.Title.map(dict(Mr='Mr', Miss = 'Miss', Mrs = 'Mrs', Master = 'Master'))

    add_features(df)

In [36]:
splits = RandomSplitter(seed=42)(df)

In [42]:
dls = TabularPandas(
    df, splits=splits,
    procs = [Categorify, FillMissing, Normalize],
    cat_names=["Sex","Pclass","Embarked","Deck", "Title"],
    cont_names=['Age', 'SibSp', 'Parch', 'logFare', 'Cabin', 'TicketFreq', 'Family'],
    y_names="Survived", y_block = CategoryBlock(),
).dataloaders(path=".")

KeyError: "['logFare', 'TicketFreq', 'Family'] not in index"

In [28]:
learn = tabular_learner(dls, metrics = accuraccy, layers=[10,10])

NameError: name 'dls' is not defined